### List of all products 

In [16]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import json
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [17]:
import requests

BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
ENDPOINT='/products'


'''
url = BASE_URL + ENDPOINT

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}
productlist = []
listlength = 0
i = 0
for i in range(0, 20):
    query = {
        'page[number]': i,
        'page[size]': 200  # maximum
    }
    r = requests.get(url, params=query, headers=headers)
    if r.status_code == 200:
        print('Page ' + str(i) + ' loaded successfully.')

        for product in r.json()['data']:
            try:
                productlist.append(product['id'])
            except:
                pass

    if listlength == len(productlist):
        break
    #print('Product count: ' + str(len(productlist)))
    listlength = len(productlist)
    i += 1
    
'''
url = BASE_URL + ENDPOINT

query = {
  'page[number]': 1,
  'page[size]': 200
}

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}

r = requests.get(url, params=query, headers=headers)
print("Status: " + str(r.status_code))
if r.status_code == 200:
    print('Page loaded successfully.')
    print('Generated in ' + str(r.json().get('meta', {}).get('generated_in', -1)) + ' milliseconds.')
    print('Next page\'s URL is: ' + r.json()['links'].get('next', 'UNKNOWN'))
    print('Barcodes on this page:')
    for product in r.json()['data']:
        productlist = []
        for x in r.json()['data']:
            productlist.append(x['id'])
print(productlist)
#'''

Status: 200
Page loaded successfully.
Generated in 191 milliseconds.
Next page's URL is: https://www.openfood.ch/api/v3/products?page%5Bnumber%5D=2&page%5Bsize%5D=200
Barcodes on this page:
[1000, 10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 1001, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017, 10018, 10019, 1002, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 1003, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 1004, 10040, 10041, 10042, 10043, 10044, 10045, 10046, 10047, 10048, 10049, 1005, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071, 10072, 10073, 10074, 10075, 10076, 10077, 10078, 10079, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098, 10099, 1010, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107, 10108, 1010

### Getting list of useful characteristics from id

In [18]:
%%time
BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
dInfo = {}
dIngredients = {}
dNutrients = {}

for PRODUCT_ID in productlist:
    ENDPOINT='/products/{}'.format(PRODUCT_ID)

    url = BASE_URL + ENDPOINT

    headers = {
      'Authorization': 'Token token=' + KEY,
      'Accept': 'application/json',
      'Content-Type': 'application/vnd.api+json',
      'Accept-Encoding': 'gzip,deflate'
    }

    r = requests.get(url, headers=headers)
    #print ('Status: ' + str(r.status_code))
    item = {}
    item_ing = {}
    item_nut = {}
    data = r.json()['data']
    if r.status_code == 200:
        try:
            item['name'] = item_ing['name'] = item_nut['name'] = data['display_name_translations']['fr']
            item['id'] = item_ing['id'] = item_nut['id'] = data['id']
            #item['barcode'] = item_ing['barcode'] = item_nut['barcode'] = data['barcode']
            
            item['quantity'] = data['quantity']
            item['unit'] = data['unit']
            item['portion'] = data['portion_quantity']
            item['portion unit'] = data['portion_unit']
            item['alcohol'] = data['alcohol_by_volume']
            
            item_ing['ingredients'] = data['ingredients_translations']['fr']
            
            nuts = {}
            for nut in data['nutrients'].keys():
                nuts[nut] = {'per_day':data['nutrients'][nut]['per_day'],
                             'per_hundred':data['nutrients'][nut]['per_hundred'],
                             'per_portion':data['nutrients'][nut]['per_portion']}
            item_nut['nutrients'] = nuts
            
            dInfo[data['barcode']] = item    
            dIngredients[data['barcode']] = item_ing
            dNutrients[data['barcode']] = item_nut
        except:
            pass

CPU times: user 10.2 s, sys: 392 ms, total: 10.6 s
Wall time: 46.3 s


### Translate ingredients

In [ ]:
%%time
from watson_developer_cloud import LanguageTranslatorV2 as LanguageTranslator

language_translator = LanguageTranslator(username='a69f77dc-9248-4069-85b5-1ac1ef6bfd7d',password='UQznP8xcKtuq')

count = 0
remove = []
for barcode, tmp in dIngredients.items():
    try:        
        ingredients_ = ' '.join(tmp['ingredients'][1].splitlines())
        language_json = language_translator.identify(ingredients_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dIngredients[barcode]['ingredients'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dIngredients[barcode]['ingredients'] = "en_kept, " + ingredients_       
        name_ = ' '.join(tmp['name'][1].splitlines())
        language_json = language_translator.identify(name_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dInfo[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
            dIngredients[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
            dNutrients[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dInfo[barcode]['name'] = "en_kept, " + name_
            dIngredients[barcode]['name'] = "en_kept, " + name_
            dNutrients[barcode]['name'] = "en_kept, " + name_
                        
    except:
        remove.append(barcode)
        
    count += 1
    if count % 50 == 0:
        print("%s/%s items done!"%(count, len(dIngredients)))

# remove the ones (from everywhere) which couldn't be translated
for barcode in remove:
    del dInfo[barcode]
    del dIngredients[barcode]
    del dNutrients[barcode]

50 done!
100 done!
150 done!


In [ ]:
# dump dictionaries to json files (load it in with `json.load(fName)`)

with open("dInfo.json", "w") as f_:
    json.dump(dInfo, f_)
with open("dIngredients.json", "w") as f_:
    json.dump(dIngredients, f_)
with open("dNutrients.json", "w") as f_:
    json.dump(dNutrients, f_)